In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.1 MB/s eta 0:00:00


In [4]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer

tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

In [5]:
paragraphs = ['a', 'b', 'c', 'd']
questions = ['a', 'b', 'x', 'p']

In [6]:
question_embeddings = []
for question in questions:
    input_ids = tokenizer(question, return_tensors="pt")["input_ids"]
    question_embeddings.append(model(input_ids).pooler_output.detach().numpy())

In [7]:
paragraph_embeddings = []
for paragraph in paragraphs:
    input_ids = tokenizer(paragraph, return_tensors="pt")["input_ids"]
    paragraph_embeddings.append(model(input_ids).pooler_output.detach().numpy())

In [8]:
THRESHOLD = 0.6
results = []

for i in range(len(questions)):
    question_embedding = question_embeddings[i]
    x = {
        "question":questions[i],
        "question_embedding":question_embedding
    }
    similarity_scores = []
    for j in range(len(paragraphs)):
        paragraph_embedding = paragraph_embeddings[j]
        sim = cosine_similarity(paragraph_embedding, question_embedding)
        similarity_scores.append(sim)
    
    max_sim = max(similarity_scores)
    if max_sim >= THRESHOLD:
        index = similarity_scores.index(max_sim)
        x["answerable"] = True
        x["paragraph"] = paragraphs[index]
        x["paragraph_embedding"] = paragraph_embeddings[index]